In [1]:
import torch

In [2]:
def load_pretrained_tacotron_model(weights_file = None):
    if weights_file is None:
        pretrained_tacotron2 = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp32', weights_only=True, pretrained=True)
        pretrained_tacotron2 = pretrained_tacotron2.to('cuda')
        pretrained_tacotron2.eval()
        return pretrained_tacotron2
    else:
        tacotron2 = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp32', weights_only=True, pretrained = False)
        tacotron2.load_state_dict(torch.load(weights_file, weights_only=True))
        tacotron2 = tacotron2.to('cuda')
        tacotron2.eval()
        return tacotron2


In [3]:
def load_hifigan():
    hifigan, vocoder_train_setup, denoiser = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_hifigan')
    hifigan = hifigan.to('cuda')
    hifigan.eval()
    return hifigan

In [4]:
def load_text_processing_utils():
    return torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')

In [5]:
utils = load_text_processing_utils()
hifigan = load_hifigan()

tacotron2_pretrained = load_pretrained_tacotron_model()
tacotron2_finetuned_english10 = load_pretrained_tacotron_model("./tacotron2_finetuned_english10.pt")
tacotron2_finetuned_french38 = load_pretrained_tacotron_model("./tacotron2_finetuned_french38.pt")

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/SpeechSynthesis/HiFiGAN/hifigan/entrypoints.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during 

HiFi-GAN: Removing weight norm.


Using cache found in /root/.cache/torch/hub/nvidia_DeepLearningExamples_torchhub
/root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/SpeechSynthesis/Tacotron2/tacotron2/entrypoints.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please op

In [6]:

texts = [
    "All the world's a stage, all the men and women merely players",
    "To be, or not to be, that is the question",
    "I am constant, as the northern star",
    "Vaulting ambition, which o’erleaps itself",
    "Tomorrow, and tomorrow, and tomorrow, creeps from day to day",
    "The wheel is come full circle",
    "The better part of valour is discretion",
    "The quality of mercy is not strained",
]

inputs = [utils.prepare_input_sequence([text]) for text in texts]

In [8]:
from IPython import display
from ipywidgets import Output, GridspecLayout


grid = GridspecLayout(len(texts) + 1, 4)


for i, text in enumerate(["Sentence", "Tacotron2", "Tacotron2 Finetuned 1", "Tacotron2 Finetuned 2"]):
    out = Output()
    with out:
        print(text)
    grid[0, i] = out

for i in range(len(texts)):

    out = Output()
    with out:
        print(texts[i])
    grid[i + 1, 0] = out

    with torch.inference_mode():
        mel, _, _ = tacotron2_pretrained.infer(inputs[i][0], inputs[i][1])
        audios = hifigan(mel).float()
        audios = audios.squeeze(1) * 32768.0
        audio_numpy = audios[0].cpu().numpy()
        out = Output()
        with out:
            display.display(display.Audio(audio_numpy, rate=22050))
            print(str(round(audio_numpy.shape[0] / 22050.0, 2)) + " sec\n")
        grid[i + 1, 1] = out

    with torch.inference_mode():
        mel, _, _ = tacotron2_finetuned_english10.infer(inputs[i][0], inputs[i][1])
        audios = hifigan(mel).float()
        audios = audios.squeeze(1) * 32768.0
        audio_numpy = audios[0].cpu().numpy()
        out = Output()
        with out:
            display.display(display.Audio(audio_numpy, rate=22050))
            print(str(round(audio_numpy.shape[0] / 22050.0, 2)) + " sec\n")
        grid[i + 1, 2] = out

    with torch.inference_mode():
        mel, _, _ = tacotron2_finetuned_french38.infer(inputs[i][0], inputs[i][1])
        audios = hifigan(mel).float()
        audios = audios.squeeze(1) * 32768.0
        audio_numpy = audios[0].cpu().numpy()
        out = Output()
        with out:
            display.display(display.Audio(audio_numpy, rate=22050))
            print(str(round(audio_numpy.shape[0] / 22050.0, 2)) + " sec\n")
        grid[i + 1, 3] = out

grid

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…